In [3]:
%matplotlib inline
import matplotlib
import numpy as np
import omni_decode as omni

sample_rate = 2048000      # sample rate of sdr
samples_per_bit = 50.4185  # This was computed based on previously seen data.

samples = np.fromfile('fsksignal.dat',dtype=np.float32)
packets_offsets = omni.find_offsets(samples,samples_per_bit)

for po in packets_offsets:
    packet_samples = samples[slice(*po)]
    bytes = omni.decode_packet(packet_samples, samples_per_bit)
    print "".join([format(n, '02x') for n in bytes])


5756783e0290577e3e02905660061c020005de08
5756783e029057c03e02905668143a30004c8000000157ff06b76a
5756783e029056823e02905792
